In [ ]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['spending_db']['spending'].aggregate([
    {
        '$project': {
            '_id': 1,
            'ad_creation_time': 1,
            'ad_creative_bodies': 1,
            'ad_creative_link_titles': 1,
            'ad_delivery_start_time': 1,
            'ad_delivery_stop_time': 1,
            'beneficiary_payers': 1,
            'bylines': 1,
            'currency': 1,
            'delivery_by_region': {
                '$filter': {
                    'input': '$delivery_by_region',
                    'as': 'region',
                    'cond': {
                        '$in': [
                            '$$region.region', [
                                'Hradec Králové Region', 'Prague', 'South Moravian Region', 'South Bohemian Region', 'Vysočina Region', 'Karlovy Vary Region', 'Liberec Region', 'Olomouc Region', 'Moravian-Silesian Region', 'Pardubice Region', 'Plzeň Region', 'Central Bohemian Region', 'Ústí nad Labem Region', 'Zlín Region'
                            ]
                        ]
                    }
                }
            },
            'demographic_distribution': {
                '$filter': {
                    'input': '$demographic_distribution',
                    'as': 'demographics',
                    'cond': {
                        '$and': [
                            {
                                '$in': [
                                    '$$demographics.gender', [
                                        'female', 'male'
                                    ]
                                ]
                            }, {
                                '$ne': [
                                    '$$demographics.age', 'Unknown'
                                ]
                            }
                        ]
                    }
                }
            },
            'target_locations': 1,
            'target_gender': 1,
            'target_ages': 1,
            'estimated_audience_size': 1,
            'impressions': 1,
            'languages': 1,
            'page_id': 1,
            'page_name': 1,
            'publisher_platforms': 1,
            'spend': 1,
            'eu_total_reach': 1,
            'age_country_gender_reach_breakdown': 1
        }
    }, {
        '$set': {
            'age_gender_breakdowns': {
                '$first': '$age_country_gender_reach_breakdown.age_gender_breakdowns'
            }
        }
    }, {
        '$unset': 'age_country_gender_reach_breakdown'
    }, {
        '$addFields': {
            'publisher_platforms': {
                '$reduce': {
                    'input': '$publisher_platforms',
                    'initialValue': '',
                    'in': {
                        '$concat': [
                            '$$value', {
                                '$cond': [
                                    {
                                        '$eq': [
                                            '$$value', ''
                                        ]
                                    }, '', ','
                                ]
                            }, '$$this'
                        ]
                    }
                }
            },
            'target_location_name': {
                '$arrayElemAt': [
                    '$target_locations.name', 0
                ]
            },
            'target_location_type': {
                '$arrayElemAt': [
                    '$target_locations.type', 0
                ]
            },
            'target_ages_bot': {
                '$arrayElemAt': [
                    '$target_ages', 0
                ]
            },
            'target_ages_top': {
                '$arrayElemAt': [
                    '$target_ages', 1
                ]
            },
            'payer': {
                '$arrayElemAt': [
                    '$beneficiary_payers.payer', 0
                ]
            },
            'beneficiary': {
                '$arrayElemAt': [
                    '$beneficiary_payers.beneficiary', 0
                ]
            },
            'languages': {
                '$arrayElemAt': [
                    '$languages', 0
                ]
            },
            'ad_creative_bodies': {
                '$arrayElemAt': [
                    '$ad_creative_bodies', 0
                ]
            },
            'average_audience_size': {
                '$divide': [
                    {
                        '$add': [
                            {
                                '$toInt': '$estimated_audience_size.lower_bound'
                            }, {
                                '$toInt': {
                                    '$ifNull': [
                                        '$estimated_audience_size.upper_bound', '$estimated_audience_size.lower_bound'
                                    ]
                                }
                            }
                        ]
                    }, 2
                ]
            },
            'average_spend': {
                '$divide': [
                    {
                        '$add': [
                            {
                                '$toInt': '$spend.lower_bound'
                            }, {
                                '$toInt': {
                                    '$ifNull': [
                                        '$spend.upper_bound', '$spend.lower_bound'
                                    ]
                                }
                            }
                        ]
                    }, 2
                ]
            },
            'average_impressions': {
                '$divide': [
                    {
                        '$add': [
                            {
                                '$toInt': '$impressions.lower_bound'
                            }, {
                                '$toInt': {
                                    '$ifNull': [
                                        '$impressions.upper_bound', '$impressions.lower_bound'
                                    ]
                                }
                            }
                        ]
                    }, 2
                ]
            }
        }
    }, {
        '$unset': [
            'target_locations', 'target_ages', 'estimated_audience_size', 'spend', 'impressions', 'beneficiary_payers'
        ]
    }
])